### This notebook acts as an prototype in building the model Vocab

In [3]:
with open('input.txt', 'r') as f:
    text = f.read()

In [2]:
print("len of the dataset in a characters: ", len(text))

len of the dataset in a characters:  1115393


In [10]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### Converting Raw text to sequence of Intergers

In [ ]:
# create a mapping of characters to integers
stoi = { ch:i for i, ch in enumerate(chars) } # string to integer
itos = { i:ch for i, ch in enumerate(chars) } # integer to string

encode = lambda s: [stoi[c] for c in s] # encoder : take a string, output a list of integer
decode = lambda l: ''.join([itos[i] for i in l]) # decoder : take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


### Tokenization, train/val split

In [9]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [12]:
# train and test splits:

n = int(0.9 * len(data)) # first 90% will be train, rest 10% will be test
train_data = data[:n]
test_data = data[n:]

In [14]:
block_size = 8
train_data[:block_size + 1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
# this is a code to illustrate the concept of block size in a language model

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t + 1]
    target  = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process parallel???
block_size = 8 # what is the maximum context length for predictions ???

def get_batch(split):
    # generate a small batch of data of inputs x and y targets:
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size))
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i+block_size] for i in ix])
    
    return x, y